In [1]:
import torch 
import os 
from subprocess import call
from matplotlib.pyplot import imshow
from miscc.config import cfg,cfg_from_file
import pprint
from miscc.utils import weights_init,mkdir_p
import torchfile
import numpy as np 
from torch.autograd import Variable
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt 

cfg_from_file("cfg/demo.yml")
cfg.GPU_ID = '0'
cfg.NET_G = '../models/birds/netG_epoch_350.pth'
model="birds"
s_gpus = cfg.GPU_ID.split(',')
gpus = [int(ix) for ix in s_gpus]
torch.cuda.set_device(gpus[0])
pprint.pprint(cfg)

{'CONFIG_NAME': 'stageII',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': '../data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'CONDITION_DIM': 128, 'DF_DIM': 96, 'GF_DIM': 192, 'R_NUM': 2},
 'GPU_ID': '0',
 'IMSIZE': 256,
 'NET_D': '',
 'NET_G': '../models/birds/netG_epoch_350.pth',
 'STAGE': 2,
 'STAGE1_G': '',
 'TEXT': {'DIMENSION': 1024},
 'TRAIN': {'BATCH_SIZE': 16,
           'COEFF': {'KL': 2.0},
           'DISCRIMINATOR_LR': 0.0002,
           'FLAG': False,
           'GENERATOR_LR': 0.0002,
           'LR_DECAY_EPOCH': 600,
           'MAX_EPOCH': 600,
           'PRETRAINED_EPOCH': 600,
           'PRETRAINED_MODEL': '',
           'SNAPSHOT_INTERVAL': 50},
 'VIS_COUNT': 64,
 'WORKERS': 4,
 'Z_DIM': 100}


In [2]:
def load_network_stageII():
        from model import STAGE1_G, STAGE2_G, STAGE2_D

        Stage1_G = STAGE1_G()
        netG = STAGE2_G(Stage1_G)
        netG.apply(weights_init)
        
        if cfg.NET_G != '':
            state_dict = \
                torch.load(cfg.NET_G,
                           map_location=lambda storage, loc: storage)
            netG.load_state_dict(state_dict)
            print('Load from: ', cfg.NET_G)
        elif cfg.STAGE1_G != '':
            state_dict = \
                torch.load(cfg.STAGE1_G,
                           map_location=lambda storage, loc: storage)
            netG.STAGE1_G.load_state_dict(state_dict)
            print('Load from: ', cfg.STAGE1_G)
        else:
            print("Please give the Stage1_G path")
            return

        netD = STAGE2_D()
        netD.apply(weights_init)
        if cfg.NET_D != '':
            state_dict = \
                torch.load(cfg.NET_D,
                           map_location=lambda storage, loc: storage)
            netD.load_state_dict(state_dict)
            print('Load from: ', cfg.NET_D)
        

        if cfg.CUDA:
            netG.cuda()
            netD.cuda()
        return netG, netD
    

In [3]:
def sample(datapath):

        netG, _ = load_network_stageII()
        netG.eval()

        # Load text embeddings generated from the encoder
        t_file = torchfile.load(datapath)
        captions_list = t_file.raw_txt
        embeddings = np.concatenate(t_file.fea_txt, axis=0)
        num_embeddings = len(captions_list)
        print('Successfully load sentences from: ', datapath)
        print('Total number of sentences:', num_embeddings)
        print('num_embeddings:', num_embeddings, embeddings.shape)
        # path to save generated samples
        save_dir = cfg.NET_G[:cfg.NET_G.find('.pth')]
        mkdir_p(save_dir)

        batch_size = cfg.TRAIN.BATCH_SIZE
        nz = cfg.Z_DIM
        noise = Variable(torch.FloatTensor(batch_size, nz))
        if cfg.CUDA:
            noise = noise.cuda()
        count = 0
        while count < num_embeddings:
            if count > 3000:
                break
            iend = count + batch_size
            if iend > num_embeddings:
                iend = num_embeddings
                count = num_embeddings - batch_size
            embeddings_batch = embeddings[count:iend]
            # captions_batch = captions_list[count:iend]
            txt_embedding = Variable(torch.FloatTensor(embeddings_batch))
            if cfg.CUDA:
                txt_embedding = txt_embedding.cuda()

            noise.data.normal_(0, 1)
            inputs = (txt_embedding, noise)
            _, fake_imgs, mu, logvar = \
                nn.parallel.data_parallel(netG, inputs,gpus)
            file = open("captions_birds.txt","r")
            lines =   file.readlines()
            for i in range(batch_size): 
                save_name = '%s/%d.png' % (save_dir, count + i)
                im = fake_imgs[i].data.cpu().numpy()
                im = (im + 1.0) * 127.5
                im = im.astype(np.uint8)
                # print('im', im.shape)
                im = np.transpose(im, (1, 2, 0))
                # print('im', im.shape)
                im = Image.fromarray(im)
                plt.figure()
                plt.title(lines[i])
                plt.imshow(im)
            count += batch_size

In [6]:
if "captions_birds_.txt" in os.listdir("."):
    os.remove("captions_birds_.txt")
line = input("")
with open('captions_birds_.txt', 'a') as the_file:
    the_file.write(line)

this bird has blue body and yellow beak.


In [7]:
if "results.t7" in os.listdir("."):
    os.remove("results.t7")
file = open("captions_birds_.txt","r")
lines =   file.readlines()
call(["th","get_embedding.lua"])
sample("results.t7")

Load from:  ../models/birds/netG_epoch_350.pth
Successfully load sentences from:  results.t7
Total number of sentences: 1
num_embeddings: 1 (1, 1024)


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 1 and 16 in dimension 0 at /pytorch/aten/src/THC/generic/THCTensorMath.cu:111